In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from rich import print

In [2]:
from util import  *
embedder = get_embedder["m3"](device_map="cuda:0")


run on 10.10.66.142, /data/qwtang


2025-05-20 14:49:18.332 | INFO     | gmpa.embed:__init__:21 - load st_embedder done!


In [3]:
llm = get_llm["qwen_7b"](device_map="cuda:0")


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [4]:
from gmpa import Rag
rag = Rag(embedder, llm)


In [5]:
path = "data/我只知道人是什么 (余华).txt"
with open(path) as f:
    text = f.read()
len(text)

108347

In [10]:
rag.build(
    document=text,
    chinese=True,
    enable_large_chunk_summary=True,
)

Build layers:   0%|          | 0/2 [00:00<?, ?it/s]

Large chunks summary:   0%|          | 0/305 [00:00<?, ?it/s]

2025-05-20 15:28:07.965 | INFO     | gmpa.db:from_document:159 - large chunks info:{'len': 305, 'max': 1251, 'avg': 421.56, 'min': 269, 'token_max': 587, 'token_avg': 482.85, 'token_min': 249}
2025-05-20 15:28:08.045 | INFO     | gmpa.db:from_list:191 - chunks info:{'len': 1951, 'max': 1251, 'avg': 121.36, 'min': 4, 'token_max': 587, 'token_avg': 141.7, 'token_min': 4}
2025-05-20 15:28:08.049 | INFO     | gmpa.db:from_list:196 - encoding
2025-05-20 15:28:11.625 | INFO     | gmpa.db:from_list:202 - indexing
2025-05-20 15:28:11.632 | INFO     | gmpa.db:from_list:215 - {'C': {1951}, 'E': {1951}, 'I': {1951}}
2025-05-20 15:28:11.633 | INFO     | gmpa.db:build_graph_leiden:330 - init_resolution is not None
2025-05-20 15:28:11.873 | INFO     | gmpa.db:build_graph_leiden:347 - G(V, E) = (1951, 17526)
2025-05-20 15:28:11.947 | INFO     | gmpa.db:build_graph_leiden:362 - communities l, m, n= 23,214,1
2025-05-20 15:28:11.952 | INFO     | gmpa.db:build_graph_leiden:374 - community_context l, m, n

In [ ]:
question="这本书的主要内容是什么?"
context=rag.retrive(question)
print(context)

[
    '目录\nCONTENTS\n我只知道人是什么 \n爸爸出差时 \n我们与他们 \n我与东亚 \n我的书游荡世界的经历 
\n民族传统中的特别性格 \n好的译文应该和原作打成平局 \n我的三个现实和梦想',
    '，四卷本的书读了两遍这是什么样的吸引力',
    '。它提醒读者，人生是复杂多变的，一个人在成长过程中，会遇到诸多困难与挫折，同时也会有诸多收获',
    '我读完这本书有一个感觉，这是一个江湖中人写出来的书，是一个经历了很多的人才能写出来的书。',
    '我的书游荡世界的经历'
]

In [ ]:
promplate="""Given the Question, Context below, provide a logical reasoning to get the answer. Please use the format of: ##Reason: <reason> ##Answer: <answer>.

Question: {question}

Context: {context}"""
prompt=promplate.format(question=question, context=context)
print(prompt)

Given the Question, Context below, provide a logical reasoning to get the answer. Please use the format of: 
##Reason: <reason> ##Answer: <answer>.

Question: 这本书的主要内容是什么?

Context: ['目录\nCONTENTS\n我只知道人是什么 \n爸爸出差时 \n我们与他们 \n我与东亚 \n我的书游荡世界的经历 
\n民族传统中的特别性格 \n好的译文应该和原作打成平局 \n我的三个现实和梦想', 
'，四卷本的书读了两遍这是什么样的吸引力', 
'。它提醒读者，人生是复杂多变的，一个人在成长过程中，会遇到诸多困难与挫折，同时也会有诸多收获', 
'我读完这本书有一个感觉，这是一个江湖中人写出来的书，是一个经历了很多的人才能写出来的书。', '我的书游荡世界的经历']

In [22]:
answer=llm.answer_question(prompt)
print(answer)

##Reason: 根据提供的目录和评论信息，可以推断出这本书的内容主要涉及个人成长经历、文化观察以及对翻译的看法等。
##Answer: 
本书主要内容包括作者的成长经历、个人对不同文化的观察和思考，以及对翻译工作的见解。书中还提到了作者的书在全球范围内
的传播经历，并强调了人生的复杂性和多变性。